In [38]:
! pip install python-mnist pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 9.0 MB/s eta 0:00:0000:0100:01


In [57]:
from mnist import MNIST
import numpy as np
import json
import pyarrow as pa
import pyarrow.parquet as pq

In [8]:
mndata = MNIST("/Users/rorywaite/code/mnist_rs/data")
images, labels = mndata.load_training()
training_set = np.array(images)
training_labels = np.array(labels)

In [40]:
num_classes = 10
num_directions = 2
num_images, img_size = np.shape(training_set)

In [54]:
param = np.ones(shape=[num_classes * img_size]).astype('float64')
rs = np.random.RandomState(42)
np.random.set_state(rs.get_state())
directions = np.random.random_sample(size=[num_directions, img_size * num_classes]).astype('float64')
projection = np.concatenate([directions, np.expand_dims(param, 0)], axis=0)
# Because the feature vector is mostly sparse, we reshape our projection matrix
reshaped = projection.transpose().reshape([num_classes, img_size, -1])
projected = training_set @ reshaped


float64
float64


In [58]:
polytopes = projected.transpose([1,0,2]).reshape([-1])
polytope_dim = num_classes * (num_directions + 1)
vertex_index = np.tile(np.arange(stop=polytope_dim, dtype="i"), num_images)
polytope_index = np.arange(stop=polytope_dim * num_images, dtype="i") // polytope_dim

polytope_table = pa.table({"polytope": polytope_index, "vertex": vertex_index, "value": polytopes})
pq.write_table(polytope_table, "mnist.parquet")
# inputs = [{"vertices" : points.tolist()} for points in polytopes]
# print(inputs[0])
# with open("mnist.json", "wt") as out_file:
#     json.dump(inputs, out_file, indent=2)